In [1]:
# To Match Karst to Tile
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, timedelta

from shapely.geometry import Point, Polygon
import geopandas as gpd

# my script
from w210_attribute_library_scale import tilekey, haversine_distance, withinstates

datdir = "../data/"
attrs = "../attrs/"
modeld = "../model/"

/Users/carlosmoreno/opt/anaconda3/envs/w210/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Read Tile Data

In [4]:
tiled = pd.read_csv(attrs+"scale_up_F_tile_soil_attr.csv")
tiled.head(1)

,Key,Key_y,date_ws,name,lon_t,lat_t,lon_w,lat_w,Distance,rolling_7_precip,...,County,Center Latitude,Center Longitude,Calcium Carbonate,Gypsum,Soil Health Organic Matter,Percent Clay,Percent Sand,Percent Silt,Available Water Storage
0,033_174,720383_53847,2021-06-01,WHITING FIELD NAVAL AIR STATI,-87.624327,30.880381,-87.023,30.704,25.161924,0.63,...,Escambia_FL033,30.611664,-87.33904,0.0,0,6.177,10.1146,57.0835,8.9811,18.52059


In [5]:
tiled.columns

Index(['Key', 'Key_y', 'date_ws', 'name', 'lon_t', 'lat_t', 'lon_w', 'lat_w',
       'Distance', 'rolling_7_precip', 'rolling_15_precip',
       'rolling_30_precip', 'rolling_60_precip', 'rolling_90_precip',
       'y1_mean_prc', 'y1_max_prc', 'y1_mean_tmp', 'y1_max_tmp', 'y1_min_tmp',
       'y2_mean_prc', 'y2_max_prc', 'y2_mean_tmp', 'y2_max_tmp', 'y2_min_tmp',
       'y3_mean_prc', 'y3_max_prc', 'y3_mean_tmp', 'y3_max_tmp', 'y3_min_tmp',
       'County', 'Center Latitude', 'Center Longitude', 'Calcium Carbonate',
       'Gypsum', 'Soil Health Organic Matter', 'Percent Clay', 'Percent Sand',
       'Percent Silt', 'Available Water Storage'],
      dtype='object')

In [6]:
tiled = tiled[['Key', 'Calcium Carbonate',
       'Gypsum', 'Soil Health Organic Matter', 'Percent Clay', 'Percent Sand',
       'Percent Silt', 'Available Water Storage']]
tiled.head(1)

,Key,Calcium Carbonate,Gypsum,Soil Health Organic Matter,Percent Clay,Percent Sand,Percent Silt,Available Water Storage
0,033_174,0.0,0,6.177,10.1146,57.0835,8.9811,18.52059


In [7]:
tilesat = pd.read_csv(attrs+"scale_up_F_satellite_ws_attr.csv")
tilesat.columns

Index(['Key', 'date_ws', 'name_x', 'lon_t', 'lat_t', 'rolling_7_precip',
       'rolling_15_precip', 'rolling_30_precip', 'rolling_60_precip',
       'rolling_90_precip', 'y1_mean_prc', 'y1_max_prc', 'y1_mean_tmp',
       'y1_max_tmp', 'y1_min_tmp', 'y2_mean_prc', 'y2_max_prc', 'y2_mean_tmp',
       'y2_max_tmp', 'y2_min_tmp', 'y3_mean_prc', 'y3_max_prc', 'y3_mean_tmp',
       'y3_max_tmp', 'y3_min_tmp', 'County', 'county_fp', 'name_y', 'imagenum',
       'geometry', 'AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway',
       'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River',
       'SeaLake', 'prediction', 'prediction_name'],
      dtype='object')

In [9]:
dftile = pd.merge(tiled, tilesat, on="Key")
len(dftile)
dftile.head(1)

,Key,Calcium Carbonate,Gypsum,Soil Health Organic Matter,Percent Clay,Percent Sand,Percent Silt,Available Water Storage,date_ws,name_x,...,HerbaceousVegetation,Highway,Industrial,Pasture,PermanentCrop,Residential,River,SeaLake,prediction,prediction_name
0,033_174,0.0,0,6.177,10.1146,57.0835,8.9811,18.52059,2021-06-01,WHITING FIELD NAVAL AIR STATI,...,0.00055,3.476364e-08,1.737650e-16,1.376721e-14,3.401483e-10,6.664154e-13,0.054241,0.933093,9,SeaLake


In [11]:
dftile.columns

Index(['Key', 'Calcium Carbonate', 'Gypsum', 'Soil Health Organic Matter',
       'Percent Clay', 'Percent Sand', 'Percent Silt',
       'Available Water Storage', 'date_ws', 'name_x', 'lon_t', 'lat_t',
       'rolling_7_precip', 'rolling_15_precip', 'rolling_30_precip',
       'rolling_60_precip', 'rolling_90_precip', 'y1_mean_prc', 'y1_max_prc',
       'y1_mean_tmp', 'y1_max_tmp', 'y1_min_tmp', 'y2_mean_prc', 'y2_max_prc',
       'y2_mean_tmp', 'y2_max_tmp', 'y2_min_tmp', 'y3_mean_prc', 'y3_max_prc',
       'y3_mean_tmp', 'y3_max_tmp', 'y3_min_tmp', 'County', 'county_fp',
       'name_y', 'imagenum', 'geometry', 'AnnualCrop', 'Forest',
       'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture',
       'PermanentCrop', 'Residential', 'River', 'SeaLake', 'prediction',
       'prediction_name'],
      dtype='object')

In [12]:
dftile.to_csv(attrs+"scale_up_F_satellite_ws_soil_attr.csv", index=False)